In [1]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SequentialChain, LLMChain #importamos el SequentialChain que es el modelo completo
f = open('../api_key.txt')
api_key = f.read()
llm = ChatOpenAI(openai_api_key=api_key)

In [2]:
#Template (prompt) para soporte básico a clientes de coches
plantilla_soporte_basico_cliente = '''Eres una persona que asiste a los clientes de automóviles con preguntas básicas que pueden
necesitar en su día a día y que explica los conceptos de una manera que sea simple de entender. Asume que no tienen conocimiento
previo. Esta es la pregunta del usuario/n{input}'''

In [3]:
#Template (prompt) para soporte avanzados a nuestros expertos en mecánica
plantilla_soporte_avanzado_mecánico = '''Eres un experto en mecánica que explicas consultas avanzadas a los mecánicos
de la plantilla. Puedes asumir que cualquier que está preguntando tiene conocimientos avanzados de mecánica. 
Esta es la pregunta del usuario/n{input}'''

In [4]:
#Debemos crear una lista de diccionarios, cada diccionario contiene su nombre, la descripción (en base a la cual el enrutador
#hará su trabajo) y el prompt a usar en cada caso
prompt_infos = [
    {'name':'mecánica básica','description': 'Responde preguntas básicas de mecánicas a clientes',
     'prompt_template':plantilla_soporte_basico_cliente},
    {'name':'mecánica avanzada','description': 'Responde preguntas avanzadas de mecánica a expertos con conocimiento previo',
     'prompt_template':plantilla_soporte_avanzado_mecánico},
    
]

In [5]:
from langchain.chains import LLMChain

In [7]:
#Creamos un diccionario de objetos LLMChain con las posibles cadenas destino
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [8]:
destination_chains

{'mecánica básica': LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='Eres una persona que asiste a los clientes de automóviles con preguntas básicas que pueden\nnecesitar en su día a día y que explica los conceptos de una manera que sea simple de entender. Asume que no tienen conocimiento\nprevio. Esta es la pregunta del usuario/n{input}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000174AC61BE30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000174AC6A5B50>, root_client=<openai.OpenAI object at 0x00000174A972FAA0>, root_async_client=<openai.AsyncOpenAI object at 0x00000174AC61BE60>, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputPars

In [9]:
#Creamos el prompt y cadena por defecto puesto que son argumento obligatorios que usaremos posteriormente
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

In [10]:
#Importamos una plantilla que podremos formatear su parámetro {destinations} que tendrá cada nombre y descripción de la información de prompts
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [11]:
print(MULTI_PROMPT_ROUTER_TEMPLATE) #El parámetro importante es {destinations}, debemos formatearlo en tipo string

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:
#Creamos una string global con todos los destinos de routing usando el nombre y descripción de "prompt_infos"
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [13]:
destinations_str

'mecánica básica: Responde preguntas básicas de mecánicas a clientes\nmecánica avanzada: Responde preguntas avanzadas de mecánica a expertos con conocimiento previo'

In [14]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [15]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str #Formateamos la plantilla con nuestros destinos en la string destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), #Para transformar el objeto JSON parseándolo a una string
)

In [16]:
print(router_template) #verificamos que se ha formateado correctamente

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
mecánica básica: Responde preguntas básicas de mecánicas a clientes
mecánica avanzada: Responde pregu

In [17]:
from langchain.chains.router import MultiPromptChain

In [18]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [20]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, #El objeto con los posibles LLMChain que creamos al inicio
                         default_chain=default_chain, verbose=True #Indicamos el LLMChain por defecto (obligatorio)
                        )

In [21]:
chain.invoke("¿Cómo cambio el aceite de mi coche?")



> Entering new MultiPromptChain chain...
mecánica básica: {'input': '¿Cómo cambio el aceite de mi coche?'}
> Finished chain.


{'input': '¿Cómo cambio el aceite de mi coche?',
 'text': 'Cambiar el aceite de tu coche es una tarea importante para mantener en buen estado el motor de tu vehículo. Aquí te explico de manera sencilla cómo hacerlo:\n\n1. Primero, asegúrate de tener las herramientas necesarias, como un gato para elevar el coche, una llave para quitar el tapón del cárter de aceite, un recipiente para recoger el aceite usado, un embudo y, por supuesto, el aceite nuevo que necesitarás.\n\n2. Ubica el tapón del cárter de aceite debajo del coche. Asegúrate de que el motor esté frío para evitar quemaduras. Coloca el recipiente debajo del tapón para recoger el aceite viejo.\n\n3. Retira el tapón con la llave y deja que el aceite viejo fluya hacia el recipiente. Una vez que haya terminado de salir, cierra el tapón.\n\n4. Busca el filtro de aceite y cámbialo. Esto se puede hacer con una llave especial para filtros de aceite.\n\n5. Coloca un embudo en el orificio donde se encuentra la varilla de medición del ace

In [22]:
chain.invoke("¿Cómo funciona internamente un catalizador?")



> Entering new MultiPromptChain chain...
mecánica avanzada: {'input': '¿Cómo funciona un catalizador en el sistema de escape de un vehículo?'}
> Finished chain.


{'input': '¿Cómo funciona un catalizador en el sistema de escape de un vehículo?',
 'text': 'El catalizador es una parte fundamental en el sistema de escape de un vehículo, ya que su función principal es reducir las emisiones de gases nocivos al medio ambiente. Funciona a través de reacciones químicas que tienen lugar en su interior.\n\nEl catalizador está compuesto por metales nobles como el platino, el paladio y el rodio, que actúan como catalizadores para acelerar las reacciones químicas que convierten los gases nocivos en gases menos dañinos. Estas reacciones se producen a altas temperaturas, por lo que el catalizador se sitúa lo más cercano posible al motor para aprovechar el calor generado durante la combustión.\n\nCuando los gases de escape entran en el catalizador, los gases nocivos como el monóxido de carbono, los óxidos de nitrógeno y los hidrocarburos son convertidos en dióxido de carbono, nitrógeno y vapor de agua. Estos últimos son mucho menos perjudiciales para el medio a